In [1]:
import sys, os
from typing import Callable, List, Dict, Any
from langsmith import traceable, Client
import uuid
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:

langsmith_client = Client()

# def generate_agent_answers(
#     invoke_callback: Callable[[str], str], dataset: List[Dict[str, Any]]
# ) -> DataFrame:
#     filename = f"run_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
#     total = len(dataset)
#     print(f"🔍 Starting answer generation on {total} dataset entries...\n")
#     results = []

#     for i, entry in enumerate(dataset, start=1):
#         run_id = str(uuid.uuid4())
#         print(f"➡️  [{i}/{total}] Running entry with name: {run_id}")

#         @traceable(name=run_id, project_name="PFE")
#         def orchestrate(input: str) -> str:
#             return invoke_callback(input)

#         input: str = entry["question"]
#         reference_answer: str = entry["answer"]

#         generated_output: str = orchestrate(input=input)
#         results.append(
#             {
#                 "run_id": run_id,
#                 "input": input,
#                 "output": generated_output,
#                 "reference_answer": reference_answer,
#             }
#         )
#         pd.DataFrame(results).to_csv(filename, index=False)
#     return results
    
def benchmark_agent(results: List[Dict[str, Any]]):
    run_ids: List[str] = [i["run_id"] for i in results]
    print(f"📡 Fetching LangSmith run data for {len(run_ids)} runs...\n")
    runs=list(langsmith_client.list_runs(project_name="PFE", limit=120, execution_order=1))
    data: List[Dict[str, Any]] = []
    for run in runs:
        if run.name not in run_ids:
            continue
        duration = (
            (run.end_time - run.start_time).total_seconds()
            if run.end_time and run.start_time
            else None
        )
        d: Dict[str, Any] = {
            "run_id": run.id,
            "duration (sec)": duration,
            "input": run.inputs["input"],
            "output": run.outputs["output"],
            "error": run.error,
            "total_tokens": run.total_tokens,
        }
        data.append(d)

    print(f"📊 Assembling results into DataFrame...\n")
    df: DataFrame = pd.DataFrame(data)

    print("✅ Benchmarking complete.\n")
    return df

In [ ]:
def invoke(query: str) -> str:

    from agents.orchestrator import LLMOrchestrator

    pandas_agent=LLMOrchestrator()
    return pandas_agent.orchestrate(query)

In [4]:
dataset = pd.read_csv("../final_dataset.csv").to_dict(orient="records")

In [5]:

from concurrent.futures import ThreadPoolExecutor, as_completed

def generate_agent_answers(invoke_callback: Callable[[str], str], dataset: List[Dict[str, Any]]) -> DataFrame:
    filename = f"run_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
    total = len(dataset)
    print(f"🔍 Starting threaded answer generation on {total} dataset entries...\n")
    
    results = []

    def process_entry(entry_idx: int, entry: Dict[str, Any]) -> Dict[str, Any]:
        run_id = str(uuid.uuid4())
        print(f"➡️  [{entry_idx+1}/{total}] Running entry with name: {run_id}")
        
        @traceable(name=run_id, project_name="PFE")
        def orchestrate(input: str) -> str:
            return invoke_callback(input)

        input_str = entry["question"]
        reference_answer = entry["answer"]
        output = orchestrate(input=input_str)

        return {
            "run_id": run_id,
            "input": input_str,
            "output": output,
            "reference_answer": reference_answer,
        }

    # Use ThreadPoolExecutor to parallelize execution
    with ThreadPoolExecutor(max_workers=16) as executor:
        # Submit all jobs
        futures = [executor.submit(process_entry, i, entry) for i, entry in enumerate(dataset)]

        # Gather results as they complete
        for future in as_completed(futures):
            result = future.result()
            results.append(result)

    # Write once all threads are done
    pd.DataFrame(results).to_csv(filename, index=False)
    print(f"✅ Finished all threads. Results saved to {filename}")
    return results

In [ ]:
answers = generate_agent_answers(dataset=dataset, invoke_callback=invoke)

In [ ]:
answers

In [18]:
results_df=benchmark_agent(results=answers)

📡 Fetching LangSmith run data for 2 runs...

📊 Assembling results into DataFrame...

✅ Benchmarking complete.



In [ ]:
def summarize_test_results(df: DataFrame) -> None:
    print("📊 Summary of Test Results:\n")
    print("Total Runs:", len(df))
    print(
        "Successful Runs:",
        df["error"].fillna("No Error").value_counts().get("No Error", 0),
    )
    print("Average Duration:", df["duration (sec)"].mean(), "seconds")
    print("Total Tokens Used:", df["total_tokens"].sum(), "tokens")
    print(
        "Accuracy:", df["verdict"].value_counts(normalize=True).get(True, 0) * 100, "%"
    )

In [23]:
summarize_test_results(results_df)

📊 Summary of Test Results:

Total Runs: 2
Successful Runs: 2
Average Duration: 13.1618325 seconds
Total Tokens Used: 6722 tokens
